In [31]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
%matplotlib inline

## Get the list of champions

In [2]:
req = requests.get('https://www.basketball-reference.com/leagues/')

if req.status_code == 200:
    print('Successful request!')
    content = req.content

Successful request!


In [3]:
soup = BeautifulSoup(content, 'html.parser')
table = soup.find(name='table', attrs={'id':'stats'})

In [4]:
table_str = str(table)
df = pd.read_html(table_str, header=1)[0]

In [5]:
df.head()

,Season,Lg,Champion,MVP,Rookie of the Year,Points,Rebounds,Assists,Win Shares
0,2018-19,NBA,NaN,NaN,NaN,J. Harden (2818),A. Drummond (1232),R. Westbrook (784),J. Harden (15.2)
1,2017-18,NBA,Golden State Warriors,J. Harden,B. Simmons,L. James (2251),A. Drummond (1247),R. Westbrook (820),J. Harden (15.4)
2,2016-17,NBA,Golden State Warriors,R. Westbrook,M. Brogdon,R. Westbrook (2558),A. Drummond (1115),J. Harden (907),J. Harden (15.0)
3,2015-16,NBA,Cleveland Cavaliers,S. Curry,K. Towns,J. Harden (2376),A. Drummond (1198),R. Rondo (839),S. Curry (17.9)
4,2014-15,NBA,Golden State Warriors,S. Curry,A. Wiggins,J. Harden (2217),D. Jordan (1226),C. Paul (838),J. Harden (16.4)


As you can see, the Champion entry for the Season 2018-19 is a NaN value, we do not know yet who it is. We are right now in the 2019 NBA Finals. Let's try to train a model to predict the champion for this season.

In [6]:
champions = df[['Season', 'Champion']]
champions

,Season,Champion
0,2018-19,NaN
1,2017-18,Golden State Warriors
2,2016-17,Golden State Warriors
3,2015-16,Cleveland Cavaliers
4,2014-15,Golden State Warriors
5,2013-14,San Antonio Spurs
6,2012-13,Miami Heat
7,2011-12,Miami Heat
8,2010-11,Dallas Mavericks
9,2009-10,Los Angeles Lakers


In [7]:
year = 2018
i = 1
year_champion = dict()
while year >= 1947:
    year_champion[str(year)] = champions.iloc[i,1]
    i += 1
    year -= 1

In [8]:
print(year_champion)

{'2018': 'Golden State Warriors', '2017': 'Golden State Warriors', '2016': 'Cleveland Cavaliers', '2015': 'Golden State Warriors', '2014': 'San Antonio Spurs', '2013': 'Miami Heat', '2012': 'Miami Heat', '2011': 'Dallas Mavericks', '2010': 'Los Angeles Lakers', '2009': 'Los Angeles Lakers', '2008': 'Boston Celtics', '2007': 'San Antonio Spurs', '2006': 'Miami Heat', '2005': 'San Antonio Spurs', '2004': 'Detroit Pistons', '2003': 'San Antonio Spurs', '2002': 'Los Angeles Lakers', '2001': 'Los Angeles Lakers', '2000': 'Los Angeles Lakers', '1999': 'San Antonio Spurs', '1998': 'Chicago Bulls', '1997': 'Chicago Bulls', '1996': 'Chicago Bulls', '1995': 'Houston Rockets', '1994': 'Houston Rockets', '1993': 'Chicago Bulls', '1992': 'Chicago Bulls', '1991': 'Chicago Bulls', '1990': 'Detroit Pistons', '1989': 'Detroit Pistons', '1988': 'Los Angeles Lakers', '1987': 'Los Angeles Lakers', '1986': 'Boston Celtics', '1985': 'Los Angeles Lakers', '1984': 'Boston Celtics', '1983': 'Philadelphia 76ers

## Create the dataframe stats of champions

In [50]:
def scrape_stats(base_url, year_champion):
    champions = pd.DataFrame()
    
    for year in year_champion.keys():
        print('Extracting NBA {} champion stats'.format(year))
        if int(year) < 1950:
            base_url = 'https://www.basketball-reference.com/leagues/BAA_{}.html'
        req_url = base_url.format(year)
        req = requests.get(req_url)
        html_doc = req.text.replace('<!--', '').replace('-->', '')
        soup = BeautifulSoup(html_doc, 'html.parser')
        table = soup.find('table', attrs={'id': "team-stats-per_game"})
        df = pd.read_html(str(table), header=0)[0]
        df['Team'] = df['Team'].str[:-1]
        df['Year'] = year
        champion = df.loc[df['Team'] == year_champion[year]]
        champions = champions.append(champion)
        
    return champions

url = 'https://www.basketball-reference.com/leagues/NBA_{}.html'
df = scrape_stats(url, year_champion)

Extracting NBA 2018 champion stats
Extracting NBA 2017 champion stats
Extracting NBA 2016 champion stats
Extracting NBA 2015 champion stats
Extracting NBA 2014 champion stats
Extracting NBA 2013 champion stats
Extracting NBA 2012 champion stats
Extracting NBA 2011 champion stats
Extracting NBA 2010 champion stats
Extracting NBA 2009 champion stats
Extracting NBA 2008 champion stats
Extracting NBA 2007 champion stats
Extracting NBA 2006 champion stats
Extracting NBA 2005 champion stats
Extracting NBA 2004 champion stats
Extracting NBA 2003 champion stats
Extracting NBA 2002 champion stats
Extracting NBA 2001 champion stats
Extracting NBA 2000 champion stats
Extracting NBA 1999 champion stats
Extracting NBA 1998 champion stats
Extracting NBA 1997 champion stats
Extracting NBA 1996 champion stats
Extracting NBA 1995 champion stats
Extracting NBA 1994 champion stats
Extracting NBA 1993 champion stats
Extracting NBA 1992 champion stats
Extracting NBA 1991 champion stats
Extracting NBA 1990 

In [51]:
df.set_index('Year', inplace=True)
df.head()

,Rk,Team,G,MP,FG,FGA,FG%,3P,3PA,3P%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
Year,,,,,,,,,,,,,,,,,,,,,
2018,1.0,Golden State Warriors,82.0,240.6,42.8,85.1,0.503,11.3,28.9,0.391,...,0.815,8.4,35.1,43.5,29.3,8.0,7.5,15.5,19.6,113.5
2017,1.0,Golden State Warriors,82.0,241.2,43.1,87.1,0.495,12.0,31.2,0.383,...,0.788,9.4,35.0,44.4,30.4,9.6,6.8,14.8,19.3,115.9
2016,8.0,Cleveland Cavaliers,82.0,242.1,38.7,84.0,0.460,10.7,29.6,0.362,...,0.748,10.6,33.9,44.5,22.7,6.7,3.9,13.6,20.3,104.3
2015,1.0,Golden State Warriors,82.0,240.6,41.6,87.0,0.478,10.8,27.0,0.398,...,0.768,10.4,34.3,44.7,27.4,9.3,6.0,14.5,19.9,110.0
2014,6.0,San Antonio Spurs,82.0,240.9,40.6,83.5,0.486,8.5,21.4,0.397,...,0.785,9.3,34.0,43.3,25.2,7.4,5.1,14.4,18.2,105.4


In [52]:
df.to_csv('nba_champions2.csv')

## Get playoffs stats for all seasons for all teams and classify champion

In [114]:
def scrape_stats(base_url, year_champion):
    nba_playoffs_team_stats = pd.DataFrame()
    
    for year in year_champion.keys():
        print('Extracting NBA {} Playoffs stats'.format(year))
        if int(year) < 1950:
            base_url = 'https://www.basketball-reference.com/playoffs/BAA_{}.html'
        req_url = base_url.format(year)
        req = requests.get(req_url)
        html_doc = req.text.replace('<!--', '').replace('-->', '')
        soup = BeautifulSoup(html_doc, 'html.parser')
        table = soup.find('table', attrs={'id': "team-stats-per_game"})
        df = pd.read_html(str(table), header=0)[0]
        df['Year'] = year
        df['Champion'] = 0
        df.loc[df['Team'] == year_champion[year], 'Champion'] = 1
        nba_playoffs_team_stats = nba_playoffs_team_stats.append(df)
        
    return nba_playoffs_team_stats

url = 'https://www.basketball-reference.com/playoffs/NBA_{}.html'
df = scrape_stats(url, year_champion)

Extracting NBA 2018 Playoffs stats
Extracting NBA 2017 Playoffs stats
Extracting NBA 2016 Playoffs stats
Extracting NBA 2015 Playoffs stats
Extracting NBA 2014 Playoffs stats
Extracting NBA 2013 Playoffs stats
Extracting NBA 2012 Playoffs stats
Extracting NBA 2011 Playoffs stats
Extracting NBA 2010 Playoffs stats
Extracting NBA 2009 Playoffs stats
Extracting NBA 2008 Playoffs stats
Extracting NBA 2007 Playoffs stats
Extracting NBA 2006 Playoffs stats
Extracting NBA 2005 Playoffs stats
Extracting NBA 2004 Playoffs stats
Extracting NBA 2003 Playoffs stats
Extracting NBA 2002 Playoffs stats
Extracting NBA 2001 Playoffs stats
Extracting NBA 2000 Playoffs stats
Extracting NBA 1999 Playoffs stats
Extracting NBA 1998 Playoffs stats
Extracting NBA 1997 Playoffs stats
Extracting NBA 1996 Playoffs stats
Extracting NBA 1995 Playoffs stats
Extracting NBA 1994 Playoffs stats
Extracting NBA 1993 Playoffs stats
Extracting NBA 1992 Playoffs stats
Extracting NBA 1991 Playoffs stats
Extracting NBA 1990 

In [115]:
df.set_index('Year', inplace=True)
df.head()

,Rk,Team,G,MP,FG,FGA,FG%,3P,3PA,3P%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Champion
Year,,,,,,,,,,,,,,,,,,,,,
2018,1.0,Golden State Warriors,21,241.2,41.2,86.0,0.480,11.2,31.4,0.357,...,9.1,35.9,45.0,26.0,7.9,5.8,12.9,19.7,110.4,1
2018,2.0,Cleveland Cavaliers,22,242.3,36.9,81.9,0.450,10.0,30.3,0.331,...,9.8,31.6,41.4,19.2,6.0,4.4,12.7,19.5,101.1,0
2018,3.0,Boston Celtics,19,242.6,36.4,83.3,0.437,10.5,30.6,0.343,...,8.8,32.8,41.7,21.2,6.8,4.1,11.6,20.4,101.4,0
2018,4.0,Houston Rockets,17,240.0,37.2,85.3,0.436,13.4,39.8,0.337,...,8.8,34.2,43.0,18.8,8.1,5.5,12.1,19.1,104.9,0
2018,5.0,Utah Jazz,11,240.0,37.7,82.9,0.455,10.7,29.9,0.359,...,9.4,35.6,45.0,20.3,7.5,5.0,15.5,20.5,102.4,0


In [116]:
df.drop(columns=['Rk', 'G', 'MP'], axis=1, inplace=True)

In [117]:
df.head()

,Team,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Champion
Year,,,,,,,,,,,,,,,,,,,,,
2018,Golden State Warriors,41.2,86.0,0.480,11.2,31.4,0.357,30.0,54.6,0.550,...,9.1,35.9,45.0,26.0,7.9,5.8,12.9,19.7,110.4,1
2018,Cleveland Cavaliers,36.9,81.9,0.450,10.0,30.3,0.331,26.8,51.5,0.520,...,9.8,31.6,41.4,19.2,6.0,4.4,12.7,19.5,101.1,0
2018,Boston Celtics,36.4,83.3,0.437,10.5,30.6,0.343,25.9,52.7,0.493,...,8.8,32.8,41.7,21.2,6.8,4.1,11.6,20.4,101.4,0
2018,Houston Rockets,37.2,85.3,0.436,13.4,39.8,0.337,23.8,45.5,0.522,...,8.8,34.2,43.0,18.8,8.1,5.5,12.1,19.1,104.9,0
2018,Utah Jazz,37.7,82.9,0.455,10.7,29.9,0.359,27.0,53.0,0.509,...,9.4,35.6,45.0,20.3,7.5,5.0,15.5,20.5,102.4,0


In [129]:
nba_previous_playoffs = df.drop(columns='Team')

We are not using the Team column to not bias our model with it knowing former champions.

In [130]:
nba_previous_playoffs.head()

,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,FT,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Champion
Year,,,,,,,,,,,,,,,,,,,,,
2018,41.2,86.0,0.480,11.2,31.4,0.357,30.0,54.6,0.550,16.8,...,9.1,35.9,45.0,26.0,7.9,5.8,12.9,19.7,110.4,1
2018,36.9,81.9,0.450,10.0,30.3,0.331,26.8,51.5,0.520,17.4,...,9.8,31.6,41.4,19.2,6.0,4.4,12.7,19.5,101.1,0
2018,36.4,83.3,0.437,10.5,30.6,0.343,25.9,52.7,0.493,18.1,...,8.8,32.8,41.7,21.2,6.8,4.1,11.6,20.4,101.4,0
2018,37.2,85.3,0.436,13.4,39.8,0.337,23.8,45.5,0.522,17.2,...,8.8,34.2,43.0,18.8,8.1,5.5,12.1,19.1,104.9,0
2018,37.7,82.9,0.455,10.7,29.9,0.359,27.0,53.0,0.509,16.2,...,9.4,35.6,45.0,20.3,7.5,5.0,15.5,20.5,102.4,0


## Get stats for the NBA 2019 Season Team Stats

In [118]:
url = 'https://www.basketball-reference.com/leagues/NBA_2019.html'
req = requests.get(url)
html_doc = req.text.replace('<!--', '').replace('-->', '')
soup = BeautifulSoup(html_doc, 'html.parser')
table = soup.find('table', attrs={'id': "team-stats-per_game"})
current_season = pd.read_html(str(table), header=0)[0]
current_season['Team'] = current_season['Team'].map(lambda x: str(x)[:-1] if '*' in str(x) else str(x))
current_season['Year'] = 2019

In [119]:
current_season.set_index('Year', inplace=True)
current_season.head()

,Rk,Team,G,MP,FG,FGA,FG%,3P,3PA,3P%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
Year,,,,,,,,,,,,,,,,,,,,,
2019,1.0,Milwaukee Bucks,82,241.2,43.4,91.1,0.476,13.5,38.2,0.353,...,0.773,9.3,40.4,49.7,26.0,7.5,5.9,13.9,19.6,118.1
2019,2.0,Golden State Warriors,82,241.5,44.0,89.8,0.491,13.3,34.4,0.385,...,0.801,9.7,36.5,46.2,29.4,7.6,6.4,14.3,21.4,117.7
2019,3.0,New Orleans Pelicans,82,240.9,43.7,92.2,0.473,10.3,29.9,0.344,...,0.761,11.1,36.2,47.3,27.0,7.4,5.4,14.8,21.1,115.4
2019,4.0,Philadelphia 76ers,82,241.5,41.5,88.2,0.471,10.8,30.2,0.359,...,0.771,10.9,36.9,47.8,26.9,7.4,5.3,14.9,21.3,115.2
2019,5.0,Los Angeles Clippers,82,241.8,41.3,87.5,0.471,10.0,25.8,0.388,...,0.792,9.7,35.8,45.5,24.0,6.8,4.7,14.5,23.3,115.1


In [120]:
current_season.drop(columns=['Rk', 'G', 'MP'], axis=1, inplace=True)

In [121]:
current_season.head()

,Team,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
Year,,,,,,,,,,,,,,,,,,,,,
2019,Milwaukee Bucks,43.4,91.1,0.476,13.5,38.2,0.353,29.9,52.9,0.565,...,0.773,9.3,40.4,49.7,26.0,7.5,5.9,13.9,19.6,118.1
2019,Golden State Warriors,44.0,89.8,0.491,13.3,34.4,0.385,30.8,55.3,0.557,...,0.801,9.7,36.5,46.2,29.4,7.6,6.4,14.3,21.4,117.7
2019,New Orleans Pelicans,43.7,92.2,0.473,10.3,29.9,0.344,33.4,62.4,0.536,...,0.761,11.1,36.2,47.3,27.0,7.4,5.4,14.8,21.1,115.4
2019,Philadelphia 76ers,41.5,88.2,0.471,10.8,30.2,0.359,30.7,58.0,0.529,...,0.771,10.9,36.9,47.8,26.9,7.4,5.3,14.9,21.3,115.2
2019,Los Angeles Clippers,41.3,87.5,0.471,10.0,25.8,0.388,31.3,61.7,0.507,...,0.792,9.7,35.8,45.5,24.0,6.8,4.7,14.5,23.3,115.1


In [131]:
nba_current_season = current_season.drop(columns='Team')

In [132]:
nba_current_season.head()

,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,FT,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
Year,,,,,,,,,,,,,,,,,,,,,
2019,43.4,91.1,0.476,13.5,38.2,0.353,29.9,52.9,0.565,17.9,...,0.773,9.3,40.4,49.7,26.0,7.5,5.9,13.9,19.6,118.1
2019,44.0,89.8,0.491,13.3,34.4,0.385,30.8,55.3,0.557,16.3,...,0.801,9.7,36.5,46.2,29.4,7.6,6.4,14.3,21.4,117.7
2019,43.7,92.2,0.473,10.3,29.9,0.344,33.4,62.4,0.536,17.8,...,0.761,11.1,36.2,47.3,27.0,7.4,5.4,14.8,21.1,115.4
2019,41.5,88.2,0.471,10.8,30.2,0.359,30.7,58.0,0.529,21.2,...,0.771,10.9,36.9,47.8,26.9,7.4,5.3,14.9,21.3,115.2
2019,41.3,87.5,0.471,10.0,25.8,0.388,31.3,61.7,0.507,22.6,...,0.792,9.7,35.8,45.5,24.0,6.8,4.7,14.5,23.3,115.1


## Quick and dirty approach learned from Andrew's courses to make a first prediction

In [134]:
from sklearn.preprocessing import Imputer
imputer = Imputer(missing_values='NaN', strategy='mean', axis=0)
imputer.fit(nba_previous_playoffs)
df_arr = imputer.transform(nba_previous_playoffs)

c:\users\monikita\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [137]:
import random
random.seed(8)
feature = list(nba_previous_playoffs)
feature.remove('Champion')

In [139]:
from sklearn.preprocessing import StandardScaler

X = nba_previous_playoffs.loc[:, feature].values
X = StandardScaler().fit_transform(X)

y = np.array(nba_previous_playoffs)[:,-1]

In [142]:
from sklearn.preprocessing import StandardScaler

X = df_arr[:, :-1]
y = df_arr[:, -1]

In [143]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [153]:
from sklearn.svm import SVC
clf = SVC(kernel='linear', class_weight={1:15}, probability=True)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [154]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[115  63]
 [  1   8]]
              precision    recall  f1-score   support

         0.0       0.99      0.65      0.78       178
         1.0       0.11      0.89      0.20         9

    accuracy                           0.66       187
   macro avg       0.55      0.77      0.49       187
weighted avg       0.95      0.66      0.75       187



## Who will win the 2019 NBA Finals is the

In [164]:
prediction = clf.predict_proba(StandardScaler().fit_transform(nba_current_season))[:,1]

In [165]:
pd.DataFrame(current_season['Team'])[prediction == max(prediction)]

,Team
Year,
2019,Golden State Warriors


In [159]:
max(prediction)

0.31267606530107844

In [160]:
pd.DataFrame(current_season['Team'])[prediction == max(prediction)]

,Team
Year,
2019,Golden State Warriors
